## Code to access MGKDB 
Venkitesh ayyar
Feb 12, 2024 

In [1]:
import numpy as np
import datetime
import os

import pymongo
import pprint
from bson.objectid import ObjectId


In [7]:
import sys
sys.path.append('../src/mgkdb/support')
from mgk_login import mgk_login

## Login to NERSC database 

In [8]:
def f_login_db(method,db_credentials=None,login_dict=None):
    
    assert method in ['login_file','direct']
    
    #### Method 1: MGKDB code login.py file  with repository and saved credentials from file db_credentials
    if method=='login_file': 
        
        login = mgk_login()
        try:
            login.from_saved(os.path.abspath(db_credentials))
        except OSError:
            exit("The specified credential file is not found!")

        client = login.connect()
        
        
    #### Method 2 : Directly access database with login credentials from login_dict
    elif method=='direct': 
        
        # client = pymongo.MongoClient(login_dict['server'])[login_dict['dbname']]
        # client.authenticate(login_dict['user'],login_dict['pwd'])

        client = pymongo.MongoClient('mongodb://{user}:{pwd}@{server}:{port}/{dbname}'.format(**login_dict),directConnection=True)
        database = client[self.login['dbname']]
    
    return client

In [9]:
## Method 1 : This requires importing mgk_login.py from the MGKDB repo
db_credentials='<path_to_credentials.pkl>'

db_credentials='../../../mgkdb_data/db_credentials/ayyarv.pkl'

client = f_login_db('login_file',db_credentials,None)

In [10]:
## Method 2 
### Read-only credentials 
# login_dict={'server':<server>,
#          'port'  :'27017',
#          'dbname':<dbname>,
#          'user'  :'<db_username>',
#          'pwd'   :'<db_password>'
#         }

# client = f_login_db('direct',None,login_dict)

## Download data using client  

In [11]:
## Test extract 
print("Collections",client.list_collection_names())

print("Find Meta for OID")
for r in client.LinearRuns.find({"_id":ObjectId("5f34a519bafb0f9d07b05721")},{'Meta':1}):
    pprint.pprint(r)

## Find ids with confidence level greater than a value
print("\nFind id for confidence")
for r in client.LinearRuns.find({"Meta.confidence":{"$gt":6.0}},{"_id":1}):
    pprint.pprint(r)
    

AttributeError: 'tuple' object has no attribute 'list_collection_names'

### Explore gyrokinetics IMAS info for conversion to latest IMAS

In [18]:
print("Find gyrokinetics for OID")
for r in client.LinearRuns.find({"_id":ObjectId("66e1ba903d55a45ca8ca3701")},{'gyrokinetics':1}):
    pprint.pprint(r.keys())
    pass

Find gyrokinetics for OID
dict_keys(['_id', 'gyrokinetics'])


In [14]:
type(r),r.keys()

(dict, dict_keys(['_id', 'gyrokinetics']))

In [19]:
r['gyrokinetics'].keys()

dict_keys(['max_repr_length', 'version', 'ids_properties', 'normalizing_quantities', 'flux_surface', 'linear', 'non_linear', 'model', 'species_all', 'species', 'collisions', 'code', 'time'])

In [16]:
## Write to file 

# import yaml
# fname='gyro.yaml'

# with open(fname,'w') as f:
#     yaml.dump(r['gyrokinetics'],f,default_flow_style=False)

In [19]:
# Read from file 
# fname='gyro.yaml'

# with open(fname,'r') as f:
#     dict_2 = yaml.safe_load(f)